In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'config'

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,55.20,49,29,5.37,scattered clouds
1,1,Cap Malheureux,MU,-19.9842,57.6142,72.46,77,45,1.99,scattered clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,44.26,70,75,14.97,light intensity drizzle
3,3,Atuona,PF,-9.8000,-139.0333,78.35,74,8,15.52,clear sky
4,4,Tartagal,AR,-22.5164,-63.8013,59.70,95,100,4.59,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 0
What is your desired maximum temperature for your trip? 110


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,55.20,49,29,5.37,scattered clouds
1,1,Cap Malheureux,MU,-19.9842,57.6142,72.46,77,45,1.99,scattered clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,44.26,70,75,14.97,light intensity drizzle
3,3,Atuona,PF,-9.8000,-139.0333,78.35,74,8,15.52,clear sky
4,4,Tartagal,AR,-22.5164,-63.8013,59.70,95,100,4.59,overcast clouds
5,5,Turayf,SA,31.6725,38.6637,75.45,31,0,11.50,clear sky
6,6,Casa Nova,BR,-9.4167,-41.1333,69.08,75,7,23.96,clear sky
7,7,Ribeira Grande,PT,38.5167,-28.7000,64.99,95,69,12.55,broken clouds
8,8,Rikitea,PF,-23.1203,-134.9692,73.08,80,52,20.49,light rain
9,9,Tuktoyaktuk,CA,69.4541,-133.0374,32.63,75,35,9.28,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,55.20,49,29,5.37,scattered clouds
1,1,Cap Malheureux,MU,-19.9842,57.6142,72.46,77,45,1.99,scattered clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,44.26,70,75,14.97,light intensity drizzle
3,3,Atuona,PF,-9.8000,-139.0333,78.35,74,8,15.52,clear sky
4,4,Tartagal,AR,-22.5164,-63.8013,59.70,95,100,4.59,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
713,713,Salinopolis,BR,-0.6136,-47.3561,77.86,85,100,6.64,overcast clouds
714,714,Caucaia,BR,-3.7361,-38.6531,76.95,86,75,4.61,broken clouds
715,715,Bow Island,CA,49.8668,-111.3684,58.75,94,100,5.75,light rain
716,716,Claremore,US,36.3126,-95.6161,72.10,87,85,10.94,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bredasdorp,ZA,55.20,scattered clouds,-34.5322,20.0403,
1,Cap Malheureux,MU,72.46,scattered clouds,-19.9842,57.6142,
2,Ushuaia,AR,44.26,light intensity drizzle,-54.8000,-68.3000,
3,Atuona,PF,78.35,clear sky,-9.8000,-139.0333,
4,Tartagal,AR,59.70,overcast clouds,-22.5164,-63.8013,
5,Turayf,SA,75.45,clear sky,31.6725,38.6637,
6,Casa Nova,BR,69.08,clear sky,-9.4167,-41.1333,
7,Ribeira Grande,PT,64.99,broken clouds,38.5167,-28.7000,
8,Rikitea,PF,73.08,light rain,-23.1203,-134.9692,
9,Tuktoyaktuk,CA,32.63,scattered clouds,69.4541,-133.0374,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

NameError: name 'g_key' is not defined